# JustWalk

In [88]:
from pybehavior.models import DataSet
from pybehavior.tools import Loader, Preprocessor
import pandas as pd
import datetime

In [133]:
data = pd.read_pickle('data/walk/data7.pickle')
data.tolist()


[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [126]:
data = data.query('duration >= datetime.timedelta(minutes=5)')
data

steps        duration
user start               end                                            
1    2015-07-22 08:36:00 2015-07-22 08:44:00  103.375000 0 days 00:08:00
     2015-07-22 11:36:00 2015-07-22 11:41:00  104.800000 0 days 00:05:00
     2015-07-22 12:48:00 2015-07-22 12:57:00   96.333333 0 days 00:09:00
     2015-07-22 18:53:00 2015-07-22 19:25:00  116.468750 0 days 00:32:00
     2015-07-23 08:40:00 2015-07-23 09:13:00  113.212121 0 days 00:33:00
...                                                  ...             ...
48   2015-12-19 16:57:00 2015-12-19 17:03:00   97.166667 0 days 00:06:00
     2015-12-28 01:26:00 2015-12-28 01:32:00   94.333333 0 days 00:06:00
     2016-01-02 14:27:00 2016-01-02 14:32:00   94.800000 0 days 00:05:00
     2016-01-03 15:15:00 2016-01-03 15:20:00   94.400000 0 days 00:05:00
     2016-01-12 08:18:00 2016-01-12 08:24:00  113.166667 0 days 00:06:00

[4512 rows x 2 columns]

In [124]:
data

steps        duration
user start               end                                            
1    2015-07-22 08:36:00 2015-07-22 08:44:00  103.375000 0 days 00:08:00
     2015-07-22 08:52:00 2015-07-22 08:55:00  121.333333 0 days 00:03:00
     2015-07-22 10:56:00 2015-07-22 10:57:00  109.000000 0 days 00:01:00
     2015-07-22 11:13:00 2015-07-22 11:14:00   61.000000 0 days 00:01:00
     2015-07-22 11:23:00 2015-07-22 11:25:00   71.000000 0 days 00:02:00
...                                                  ...             ...
48   2016-01-26 12:26:00 2016-01-26 12:27:00   75.000000 0 days 00:01:00
     2016-01-26 12:57:00 2016-01-26 13:00:00   78.000000 0 days 00:03:00
     2016-01-26 15:30:00 2016-01-26 15:31:00   76.000000 0 days 00:01:00
     2016-01-26 15:33:00 2016-01-26 15:34:00   76.000000 0 days 00:01:00
     2016-01-26 16:30:00 2016-01-26 16:31:00   67.000000 0 days 00:01:00

[37771 rows x 2 columns]

In [50]:


pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)  

In [74]:
data = pd.read_pickle('data/ema/data1.pickle')

display(data[['rnum', 'created', 'order']].query('order == 0').shape)
display(data[['rnum', 'created', 'order']].drop_duplicates().query('order == 0').shape)

(3653 - 3617)/3653 * 100

(3653, 3)

(3617, 3)

0.9854913769504517

In [64]:
data = pd.read_pickle('data/ema/data3.pickle')

display(data.shape)
display(data.drop_duplicates().shape)
display(data[['rnum', 'created']].drop_duplicates().shape)
display(data[['rnum', 'created', 'question_id']].drop_duplicates().shape)




(29947, 4)

(28932, 4)

(3404, 2)

(28128, 3)

In [59]:
data = pd.read_pickle('data/ema/data4.pickle')
data.to_csv('justwalk.ema.20220920.csv')

(29947, 4)

(28932, 4)

In [49]:
data = data.pivot_table(index=['rnum', 'created'], columns='question_id', values='answer_value')
data.to_csv('data.csv')